# Treatment Recommendation Model
This notebook performs feature extraction, model training, evaluation, and saving for use in the CrewAI pipeline.

In [3]:
# Imports
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib
import os
from crewai import Agent, Task, Crew
import sys
from datetime import datetime
import json
from langchain.chat_models import ChatOpenAI

## Data Preprocessing

In [4]:
# Load the raw data
df_raw = pd.read_csv("data/prostate_patient_data.csv")
df_raw.head()

,PatientID,Clinical Notes
0,1,2020-03-13: DIAGNOSIS - Initial PSA=14.7 ng/mL...
1,2,2022-04-11: DIAGNOSIS - Initial PSA=15.4 ng/mL...
2,3,2021-09-14: DIAGNOSIS - Initial PSA=20.0 ng/mL...
3,4,"2020-10-14: DIAGNOSIS - Initial PSA=4.0 ng/mL,..."
4,5,2022-12-07: DIAGNOSIS - Initial PSA=20.0 ng/mL...


In [5]:
# Data parsing
def parse_clinical_notes(patient_id, notes_string):
    entries = notes_string.split('|')
    records = []

    for entry in entries:
        match = re.match(r"\s*(\d{4}-\d{2}-\d{2}):\s*(.*)", entry.strip())
        if not match:
            continue
        date_str, content = match.groups()
        date = datetime.strptime(date_str, "%Y-%m-%d").date()

        # Extract relevant fields
        psa_match = re.search(r"PSA=([\d.]+)", content)
        psa = float(psa_match.group(1)) if psa_match else None

        pirads_match = re.search(r"PI-RADS=(\d+)", content)
        pirads = int(pirads_match.group(1)) if pirads_match else None

        weight_match = re.search(r"Weight=([\d.]+)", content)
        weight = float(weight_match.group(1)) if weight_match else None

        bone_pain_match = re.search(r"Bone Pain=([a-zA-Z]+)", content)
        bone_pain = bone_pain_match.group(1) if bone_pain_match else None

        treatment_match = re.search(r"Treatment=([^,|]+)", content)
        treatment = treatment_match.group(1).strip() if treatment_match else None

        records.append({
            "PatientID": patient_id,
            "Date": date,
            "PSA": psa,
            "PIRADS": pirads,
            "Weight": weight,
            "BonePain": bone_pain,
            "Treatment": treatment
        })

    return records

In [6]:
# Parse all patients into structured rows
all_records = []

for _, row in df_raw.iterrows():
    patient_id = row["PatientID"]
    notes = row["Clinical Notes"]
    records = parse_clinical_notes(patient_id, notes)
    all_records.extend(records)

# Create structured DataFrame
df = pd.DataFrame(all_records)

In [7]:
df.head()

,PatientID,Date,PSA,PIRADS,Weight,BonePain,Treatment
0,1,2020-03-13,14.7,5,73.7,None,None
1,1,2020-03-13,14.7,5,73.7,Mild,ADT
2,1,2020-06-11,14.5,5,74.5,None,ADT
3,1,2020-09-09,14.4,5,75.3,Mild,ADT
4,1,2020-12-08,14.1,5,76.2,Mild,ADT


In [8]:
# Check the dataframe
df.head()

,PatientID,Date,PSA,PIRADS,Weight,BonePain,Treatment
0,1,2020-03-13,14.7,5,73.7,None,None
1,1,2020-03-13,14.7,5,73.7,Mild,ADT
2,1,2020-06-11,14.5,5,74.5,None,ADT
3,1,2020-09-09,14.4,5,75.3,Mild,ADT
4,1,2020-12-08,14.1,5,76.2,Mild,ADT


In [9]:
# Number of unique treatments
df['Treatment'].unique()

array([None, 'ADT', 'Surgery', 'Surgery + Radiation + ADT',
       'Surgery + ADT', 'Radiation', 'Radiation + ADT',
       'Surgery + Radiation'], dtype=object)

## Feature Engineering

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Function for feature engineering 
def add_engineered_features(df):
    df = df.sort_values(by=["PatientID", "Date"])

    # PSA, Weight Delta
    # Use to show the difference compared to the past visits & Include temporal information
    df["PSA_Delta"] = df.groupby("PatientID")["PSA"].diff().fillna(0)
    df["Weight_Delta"] = df.groupby("PatientID")["Weight"].diff().fillna(0)

    # How many times patient visited 
    df["VisitOrder"] = df.groupby("PatientID").cumcount() + 1

    return df

# Prepare the data
df = df.dropna(subset=["PSA", "PIRADS", "Weight", "BonePain", "Treatment"])
df["Date"] = pd.to_datetime(df["Date"])  
df["BonePainEncoded"] = df["BonePain"].map({"None": 0, "Mild": 1, "Moderate": 2, "Severe": 3})
df["Treatment"] = df["Treatment"].astype(str)
df["TreatmentEncoded"] = LabelEncoder().fit_transform(df["Treatment"])

# Apply feature engineering 
df_fe = add_engineered_features(df)

In [32]:
df_fe.head()

,PatientID,Date,PSA,PIRADS,Weight,BonePain,Treatment,TreatmentEncoded,BonePainEncoded,PSA_Delta,Weight_Delta,VisitOrder
1,1,2020-03-13,14.7,5,73.7,Mild,ADT,0,1,0.0,0.0,1
2,1,2020-06-11,14.5,5,74.5,None,ADT,0,0,-0.2,0.8,2
3,1,2020-09-09,14.4,5,75.3,Mild,ADT,0,1,-0.1,0.8,3
4,1,2020-12-08,14.1,5,76.2,Mild,ADT,0,1,-0.3,0.9,4
5,1,2021-03-08,13.9,5,76.9,Mild,ADT,0,1,-0.2,0.7,5


In [28]:
# Set Feature/Label
features = ["PSA", "PIRADS", "Weight", "BonePainEncoded", "PSA_Delta", "Weight_Delta", "VisitOrder"]
X = df_fe[features]
y = df_fe["TreatmentEncoded"]

# Train-test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

## Modeling

### 1. RandomForestClassifier

Baseline RandomForestClassifier model

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
                           precision    recall  f1-score   support

                      ADT       0.38      0.42      0.40        97
                Radiation       0.54      0.64      0.59       126
          Radiation + ADT       0.48      0.45      0.46        83
                  Surgery       0.63      0.63      0.63       123
            Surgery + ADT       0.44      0.46      0.45       102
      Surgery + Radiation       0.43      0.18      0.26        55
Surgery + Radiation + ADT       0.27      0.21      0.24        14

                 accuracy                           0.49       600
                macro avg       0.45      0.43      0.43       600
             weighted avg       0.49      0.49      0.49       600

Confusion Matrix:
[[41  3 17  4 28  0  4]
 [ 6 81  2 24 10  3  0]
 [23  3 37  2 16  2  0]
 [ 4 28  3 78  5  5  0]
 [29  5 13  2 47  2  4]
 [ 0 30  1 13  1 10  0]
 [ 4  0  4  1  1  1  3]]


RandomForestClassifier + GridSearchCV

In [33]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'class_weight': [None, 'balanced']
}

# GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    scoring='f1_weighted',
    verbose=1
)

grid_search.fit(X_train, y_train)
best_rf = grid_search.best_estimator_

# Evaluation
y_pred = best_rf.predict(X_test)
print("Best Params:", grid_search.best_params_)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Params: {'class_weight': 'balanced', 'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}
Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.41      0.39        97
           1       0.63      0.66      0.64       126
           2       0.45      0.43      0.44        83
           3       0.71      0.72      0.72       123
           4       0.44      0.47      0.46       102
           5       0.48      0.29      0.36        55
           6       0.21      0.21      0.21        14

    accuracy                           0.53       600
   macro avg       0.47      0.46      0.46       600
weighted avg       0.53      0.53      0.52       600



## 2. XGBoost
Baseline XGBoost model

In [30]:
from xgboost import XGBClassifier

# Train model
xgb_model_1 = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model_1.fit(X_train, y_train)

# Evaluate
y_pred = xgb_model_1.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

/opt/anaconda3/envs/bayes/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


                           precision    recall  f1-score   support

                      ADT       0.47      0.57      0.51        97
                Radiation       0.81      0.76      0.79       126
          Radiation + ADT       0.51      0.48      0.49        83
                  Surgery       0.79      0.88      0.83       123
            Surgery + ADT       0.53      0.50      0.52       102
      Surgery + Radiation       0.90      0.67      0.77        55
Surgery + Radiation + ADT       0.31      0.29      0.30        14

                 accuracy                           0.65       600
                macro avg       0.62      0.59      0.60       600
             weighted avg       0.66      0.65      0.65       600



In [55]:
# Save the model
import joblib
joblib.dump(xgb_model_1, "model/xgboost_model.joblib")

['model/xgboost_model.joblib']

XGBoost with hyperparameter tuning

In [47]:
xgb_model_2 = XGBClassifier(
    max_depth=5,
    learning_rate=0.5,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8
)
xgb_model_2.fit(X_train, y_train)

y_pred = xgb_model_2.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

                           precision    recall  f1-score   support

                      ADT       0.44      0.53      0.48        97
                Radiation       0.78      0.75      0.77       126
          Radiation + ADT       0.49      0.45      0.47        83
                  Surgery       0.83      0.83      0.83       123
            Surgery + ADT       0.49      0.50      0.50       102
      Surgery + Radiation       0.88      0.69      0.78        55
Surgery + Radiation + ADT       0.20      0.21      0.21        14

                 accuracy                           0.63       600
                macro avg       0.59      0.57      0.57       600
             weighted avg       0.64      0.63      0.63       600



In [52]:
pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 58.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 70.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [catboost]2/3 [catboost]
Note: you may need to restart the kernel to use updated packages.


## 3. CatBoost Model

In [53]:
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier(
    iterations=200,
    depth=5,
    learning_rate=0.05,
    verbose=0
)
cat_model.fit(X_train, y_train)
y_pred = cat_model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=le.classes_))


                           precision    recall  f1-score   support

                      ADT       0.37      0.39      0.38        97
                Radiation       0.60      0.62      0.61       126
          Radiation + ADT       0.39      0.27      0.32        83
                  Surgery       0.56      0.81      0.67       123
            Surgery + ADT       0.39      0.46      0.42       102
      Surgery + Radiation       0.14      0.02      0.03        55
Surgery + Radiation + ADT       0.50      0.14      0.22        14

                 accuracy                           0.48       600
                macro avg       0.42      0.39      0.38       600
             weighted avg       0.45      0.48      0.45       600

